# Rasterio

Geographic information systems use GeoTIFF and other formats to organize and store gridded raster datasets such as satellite imagery and terrain models. [**Rasterio**](https://rasterio.readthedocs.io/en/latest/) reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON

In [ ]:
import rasterio
from matplotlib import pyplot as plt
from rasterio.plot import show
src = rasterio.open('Products/Kigali.tiff')

In [ ]:
show(src.read())

Get informations about the product

In [ ]:
print("Width: ", src.width)
print("Height: ", src.height)
print("File name: ", src.files)
print("Is the system a projected one ?", src.crs.is_projected)
print("Projection system used: ", src.crs) # https://spatialreference.org/ref/epsg/32633/

To get the spatial coordinates of a pixel, use the dataset’s *xy()* method. The coordinates of the center of the image can be computed like this.

In [ ]:
print("Central coordinates: ", src.xy(src.height // 2, src.width // 2))
print("Coordinates of the bounding box", src.bounds)

Since the product is a geotiff, it is possible to fetch geospatial information

In [ ]:
print("The product central longitude and latitude are: lon = {0}, lat = {1}".format(src.lnglat()[0], src.lnglat()[1]))

However, collecting longitude and latitude for other parts of the products is more complicated, since every points of the image need to be converted.

Rasterio can map the pixels of a destination raster with an associated coordinate reference system and transform to the pixels of a source image with a different coordinate reference system and transform. This process is known as reprojection.

Hopefully, rasterio provides several utilities to make this processing easier. In the cells below we will : 

* Open a product with rastertio, get all the information about it's projection system
* User rasterio's utilities to reproject the GeoTIFF
* Write the GeoTIFF on the disk, in a new coordinate reference system
* Display the result

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = "EPSG:4326"  # WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS

# Open source image and get its parameters
with rasterio.open("Products/Kigali.tiff") as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update(
        {"crs": dst_crs, "transform": transform, "width": width, "height": height}
    )
    # Write output image after reprojection
    with rasterio.open(
        "Products/Kiagli.wgs84.tif",
        "w",
        **kwargs
    ) as dst:
        for i in range(1, src.count + 1): # Loop over all bands
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest,
            )

Let's open this GeoTIFF with rasterio

In [ ]:
src = rasterio.open("Products/Kiagli.wgs84.tif") # Open file
img = src.read() # Read file as a numpy array

Fetch the coordinates, to later display the image as a new layer on a map

In [ ]:
x1, y1, x2, y2 = src.bounds  # Get coordinates of image bounds
print("Coordinates of the bounding box in the EPS:4326 reference system\n")
print("Bottom left: ", (x1, y1))
print("Top right: ", (x2, y2))

In [ ]:
import folium
lon, lat = src.lnglat()  # Get longitude and latitude
m = folium.Map(location=[lat, lon], zoom_start=12)

folium.raster_layers.ImageOverlay(
    image=img[0], bounds=[[y1, x1], [y2, x2]], opacity=0.5
).add_to(m)

m

Rasterio also provides a show_hist() function for generating histograms of single or multiband rasters:

In [ ]:
from rasterio.plot import show_hist
show_hist(src, bins=50, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")